### 利用Dataframe整理收費名冊，並產出可列印的收費單
1. 讀入[收費資.xlsx]: 名冊、收費項目、收費期間 等內容
2. 整理成 [收費單_名冊.xlsx]：每個姓名使用1個工作表
3. 由 收費單_名冊.xlsx 產出 [收費單_列印.xlsx]: 加入表頭(該姓名對應的資料_年級、班級)，每個姓名使用1個工作表

In [1]:
import pandas as pd
import numpy as np

In [2]:
with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_item = pd.read_excel(xlsxFile, sheet_name='收費項目', header=0) 
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
    df_note = pd.read_excel(xlsxFile, sheet_name='註記事項', header=0)

In [3]:
df_student

,姓名,年級,班級
0,呂彥臻,一,忠
1,王大同,二,孝
2,楊小明,三,仁
3,劉大象,四,愛
4,陳中一,五,信
5,孫二,六,義


In [4]:
df_item

,項目/月份
0,費用
1,點心
2,教材/冷氣
3,共計
4,收款人


In [5]:
df_month

,月份
0,九月
1,十月
2,十一月
3,十二月
4,一月


In [6]:
df_note

,註記
0,請於拿到收費袋一週內繳費，以便學校行政事務
1,安親班收費至6:15，請家長於7:00前接小朋友，如當日有狀況，請洽蔣老師
2,若有任何問題請撥0909-413549、0926-231858 蔣老師
3,教材費每學期僅收一次，冷氣費則依當月使用狀況而定


### Transform table

In [7]:
df = df_month.T
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df

月份,九月,十月,十一月,十二月,一月


In [8]:
df = pd.concat([df_item, df], axis=1)
#df.fillna(int(0), inplace=True)
df

,項目/月份,九月,十月,十一月,十二月,一月
0,費用,NaN,NaN,NaN,NaN,NaN
1,點心,NaN,NaN,NaN,NaN,NaN
2,教材/冷氣,NaN,NaN,NaN,NaN,NaN
3,共計,NaN,NaN,NaN,NaN,NaN
4,收款人,NaN,NaN,NaN,NaN,NaN


### 收費單 名冊
#### 每位學生一個Excel工作表(所有的收費項目)

In [9]:
name_list = df_student['姓名']
i = 0
df_note_temp = pd.DataFrame()
with pd.ExcelWriter('安親班_收費單_名冊.xlsx') as writer:
    for student in name_list:
        #將註記的內容，放在column['項目/月份'] (放在收費單的第一欄位，以對齊收費單和註記的位置)
        #收費單(資料框dataframe:df) 的第一個欄位,其名稱為'項目/月份'
        df_note_temp['項目/月份'] = df_note['註記']
        df_temp = pd.concat([df, df_note_temp], axis=0)
        #產出 收費單 excel worksheet
        df_temp.to_excel(writer, sheet_name=student, index=False)
        i += 1
df_temp

,項目/月份,九月,十月,十一月,十二月,一月
0,費用,NaN,NaN,NaN,NaN,NaN
1,點心,NaN,NaN,NaN,NaN,NaN
2,教材/冷氣,NaN,NaN,NaN,NaN,NaN
3,共計,NaN,NaN,NaN,NaN,NaN
4,收款人,NaN,NaN,NaN,NaN,NaN
0,請於拿到收費袋一週內繳費，以便學校行政事務,NaN,NaN,NaN,NaN,NaN
1,安親班收費至6:15，請家長於7:00前接小朋友，如當日有狀況，請洽蔣老師,NaN,NaN,NaN,NaN,NaN
2,若有任何問題請撥0909-413549、0926-231858 蔣老師,NaN,NaN,NaN,NaN,NaN
3,教材費每學期僅收一次，冷氣費則依當月使用狀況而定,NaN,NaN,NaN,NaN,NaN


### 收費單 列印
#### 每位學生一個Excel工作表 (列印收費單)

In [10]:
i = 0
with pd.ExcelWriter('安親班_收費單_列印.xlsx') as writer:
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)
        df_tempt = pd.read_excel('安親班_收費單_名冊.xlsx', sheet_name=student, header=None)

        #將每一個學生的收費單轉成numpy array (context)
        content = df_temp.to_numpy() #轉成numpy array
        
        #放回一列：欄位名稱
        #因為 用 with..as..，包納read_excel後，header=None的功能 disable
        #所以轉成numpy arrany後少了原先欄位名稱(column lable)這一列
        content1 = np.insert(content, 0, [df_temp.columns], axis=0)
        
        #將每一個學生的收費表，加入表頭(學生年級、班級、姓名)
        content2 = np.insert(content1, 0, [np.NAN], axis=0) # numpy array加入第一列(表頭)
        content2[0, 0] = df_student.iloc[i]['年級'] + '年' +  df_student.iloc[i]['班級'] + '班  ' + student
        
        df2 = pd.DataFrame(content2) #轉成資料框(dataframe) (列印之學生收費單)
        
        #產出 列印收費單 excel worksheet，將 NAN 用 ''取代 
        df2.to_excel(writer, sheet_name=student, na_rep='', header=False, index=False)
        i += 1

### End of transforming